<figure>
  <IMG SRC="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d5/Fachhochschule_Südwestfalen_20xx_logo.svg/320px-Fachhochschule_Südwestfalen_20xx_logo.svg.png" WIDTH=250 ALIGN="right">
</figure>

# Machine Learning
### Sommersemester 2023
Prof. Dr. Heiner Giefers

## Fashion MNIST mit Keras
**Nach dem Tutorial von Google:** https://www.tensorflow.org/tutorials/keras/classification

In diesem Notebook geht es um das selbe Thema wie schon beim Aufgabenblatt zur Logistischen Regression, nämlich dem *Fashion MNIST* Datensatz.
An dieser Stelle wollen wir allerdings statt eine Multi-Klassen Logistischen Regression ein Neuronales Netz einsetzen.

Um den Code so kompakt wie möglich zu halten, verwenden wir die Keras API.
Zusätzlich benötigen Wir Funktionen aus NumPy und Matplotlib.

In [ ]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)
%load_ext tensorboard

Im Arbeitsblatt zur Logistischen Regression habe wir den Datensatz noch über eine URL aus einem Cloud Speicher heruntergeladen.
Der Fashion MNIST Datensatz ist aber ebenfalls als Standard-Beispiel über die Keras Bibliothek erhältlich.
Daher können wir ihn komfortabel über einen Keras-Aufruf herunterladen und direkt auf Trainings- und Testdatensätze aufteilen:

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

train_images = train_images.reshape(train_images.shape[0], train_images.shape[1], train_images.shape[2], 1)
test_images = test_images.reshape(test_images.shape[0], test_images.shape[1], test_images.shape[2], 1)

In [ ]:
train_labels_ohe = tf.one_hot(train_labels, depth=10)
test_labels_ohe = tf.one_hot(test_labels, depth=10)
print(test_labels_ohe[1])
print(test_labels[1])

In [ ]:
print(train_images.shape)
print(test_images.shape)

In [ ]:
plt.imshow(train_images[np.random.randint(0,train_images.shape[0])].reshape(28,28))

Die 28x28 Pixel großen Bilder bestehen aus 8-bit Grauwerten.
Um die Piwelwerte in den Bereich $[0,1]$ zu skalieren, teilen wir alle Pixel durch 255.

In [ ]:
#Pixelwerte nach [0,1] skalieren
train_images = train_images / 255.0
test_images = test_images / 255.0

Nun erzeugen wir ein sequentielles Keras Modell:

In [ ]:
#Modell definieren
model = keras.Sequential()

Zu diesem Modell können wir nun mit `model.add` Schichten hinzufügen.
Entwerfen Sie selbst eine Mehrschichtiges neuronales Netz.
Wählen Sie die Anzahl der Neuronen und die Aktivierungsfunktionen der einzelnen Schichten aus.

In [ ]:
#Modell definieren
#model.add(...

In [ ]:
#Modell definieren
model = tf.keras.Sequential()
# Must define the input shape in the first layer of the neural network
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=(28,28,1))) 
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(10, activation='softmax'))
# Take a look at the model summary
model.summary()

In [ ]:
'''
!rm ./FashionMNIST_CNN.h5
!rm -rf ./logs/*
'''

In [ ]:
import os
reuse = True
if(reuse == True and os.path.exists("FashionMNISTCNN")):
  model = keras.models.load_model("FashionMNISTCNN")
else:
  #Modell erzeugen
  model.compile(optimizer='Adam',
    loss='sparse_categorical_crossentropy',
    #loss='categorical_crossentropy',
    metrics=['accuracy'])

import datetime
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

In [ ]:
#Modell trainieren
model.fit(train_images, train_labels,
          epochs=5,
          #validation_data=(test_images, test_labels),
          callbacks=[tensorboard_callback]
         )

In [ ]:
model.save("FashionMNIST_CNN")

In [ ]:
#Trainiertes Modell auswerten
test_loss, test_acc = model.evaluate (test_images, test_labels)
print('Test accuracy:', test_acc)

In [ ]:
%tensorboard --logdir logs

Definieren Sie Ihr Modell erneut mit der Funktionalen API von Keras.

In [ ]:
#Funktionale abhängigkeiten
inputs = keras.Input(shape=(28, 28))
finputs = keras.layers.Flatten()(inputs)
l1 = keras.layers.Dense(128, activation=tf.nn.relu)(finputs)
outputs = keras.layers.Dense(10, activation=tf.nn.softmax)(l1)

#Modell definieren
model = keras.Model(inputs, outputs)

#Modell erzeugen
model.compile('sgd','sparse_categorical_crossentropy',['accuracy'])

#Modell trainieren
model.fit(train_images, train_labels, epochs=5)

#Trainiertes Modell auswerten
test_loss, test_acc = model.evaluate (test_images, test_labels)
print('Test accuracy:', test_acc)

Ein alternatives Modell mit Convolutional Layers